In [4]:
#import libraries
import pandas as pd

# 1.1

In [5]:
#download curated gene-diesase associtaions from https://www.disgenet.org/downloads and covnert tsv in csv


data = pd.read_csv('dataset/1.1/curated_gene_disease_associations.csv')
#get only data for intereted gene id C0043094
gene_disease = data.loc[data['diseaseId'] == 'C0043094']

#get list og gene symbols
list_geneSymbol = gene_disease['geneSymbol']


#download hgnc dataset from https://www.genenames.org/download/statistics-and-files/
hgnc = pd.read_csv("dataset/1.1/hgnc_complete_set.txt", sep='\t', dtype = 'unicode')
#get data (symbol,name,entrez_id,unirpot_idasdn status) for only genes stored in previous list
genes_diseases_hgnc = hgnc.loc[hgnc['symbol'].isin(list_geneSymbol)][['symbol','name','entrez_id','uniprot_ids','status']]
#get only approved data
genes_diseases_hgnc_app = genes_diseases_hgnc.loc[genes_diseases_hgnc['status']=='Approved']


#get list ofuniprotidthat we'll useto find gene descriptionin Uniprot
list_uniprotID = genes_diseases_hgnc_app['uniprot_ids']

#write in a file alll uniprot ids. This is necessary because we'll download only necessary description for these genes from uniprot 
with open('dataset/1.1/uniprot_ids.txt', 'w') as f:
    for item in list_uniprotID:
        f.write("%s\n" % item)
    f.close()

#download data containing description for every gene. We used txt file for download them
gene_description = pd.read_csv('dataset/1.1/uniprot-yourlist%3AM20201123A94466D2655679D1FD8953E075198DA817123DO.csv')
#rename column for the function of the gene
gene_description=gene_description.rename(columns={"yourlist:M20201123A94466D2655679D1FD8953E075198DA817123DO":"uniprot_ids"})


#merge two dataset. One containing all info about gene and the second one conatining description about their functions
final_genes_deseas = pd.merge(gene_description, genes_diseases_hgnc_app, on='uniprot_ids')
#rename column
final_genes_deseas=final_genes_deseas.rename(columns={"Function [CC]":"function"})
#get only first part of gene description
final_genes_deseas['Description']= final_genes_deseas.function.str[10:150]
#delete not necessary columns
final_genes_deseas = final_genes_deseas.drop(['function','Entry','status'], axis = 1) 
final_genes_deseas = final_genes_deseas[['symbol','uniprot_ids','name','entrez_id','Description']]

In [6]:
#fix column names and save xlsx file
final_genes_deseas.rename(columns={"symbol":"Gene symbol","uniprot_ids":"Uniprot AC","name":'Protein name','entrez_id':"Entrez Gene ID"})
final_genes_deseas.to_excel('dataset/1.1/1.1-gene_deseas_associations.xlsx')
final_genes_deseas.to_csv('dataset/1.1/1.1-gene_deseas_associations.csv')

# 1.2